# Visualization

This notebook provides some visualization for the instances and the solutions.

In [ ]:
from pathlib import Path

from data_schema import Instance

INSTANCE_PATH = Path("./instances/instance_20.json")

with INSTANCE_PATH.open() as f:
    instance = Instance.model_validate_json(f.read())

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import networkx as nx


def visualize_instance(instance):
    plt.figure(figsize=(10, 10))
    # Create a graph
    graph = nx.Graph()

    # Add nodes with attributes
    graph.add_node(instance.elevator_location, node_type="Elevator")
    for mine in instance.mines.values():
        graph.add_node(mine.location, node_type="Mine", ore_per_hour=mine.ore_per_hour)
    other_locations = (
        set(instance.locations)
        - {instance.elevator_location}
        - {m.location for m in instance.mines.values()}
    )
    for location in other_locations:
        graph.add_node(location, node_type="Location")

    # Add edges with weights
    for tunnel in instance.tunnels:
        graph.add_edge(
            tunnel.source,
            tunnel.target,
            weight=tunnel.throughput_per_hour,
            cost=tunnel.reinforcement_costs,
        )

    # Compute positions for visualization
    pos = nx.spring_layout(graph)

    # Define node colors and sizes
    node_colors = []
    node_sizes = []
    for node, data in graph.nodes(data=True):
        if data["node_type"] == "Elevator":
            node_colors.append("red")
            node_sizes.append(400)
        elif data["node_type"] == "Mine":
            node_colors.append("lightblue")
            node_sizes.append(
                300 + data["ore_per_hour"] * 30
            )  # Scale size by ore_per_hour
        else:
            node_colors.append("green")
            node_sizes.append(300)

    # Draw nodes and edges
    nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=node_sizes)
    nx.draw_networkx_edges(graph, pos)

    # Create a legend for node types
    legend_elements = [
        mpatches.Patch(color="red", label="Elevator"),
        mpatches.Patch(color="blue", label="Mine"),
        mpatches.Patch(color="green", label="Location"),
    ]
    plt.legend(handles=legend_elements)

    # Add node labels, including ore_per_hour for mines
    node_labels = {}
    for node, data in graph.nodes(data=True):
        if data["node_type"] == "Mine":
            node_labels[node] = f"{node}\n({data['ore_per_hour']} ore/hr)"
        else:
            node_labels[node] = str(node)
    nx.draw_networkx_labels(graph, pos, labels=node_labels, font_size=8)

    # Add edge labels for weights
    edge_labels = {
        (u, v): f"{d['weight']} (${d['cost']:.2f}K)"
        for u, v, d in graph.edges(data=True)
    }
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=8)

    plt.title(f"Budget: ${instance.budget:.2f}K")
    plt.axis("off")
    plt.show()
    return pos

In [ ]:
layout = visualize_instance(instance)

In [ ]:
from solution import MiningRoutingSolver

solver = MiningRoutingSolver(instance)
solution = solver.solve()

In [ ]:
def visualize_solution(instance, solution, pos):
    # Create a directed graph for the solution
    graph = nx.DiGraph()

    # Add nodes with attributes
    graph.add_node(instance.elevator_location, node_type="Elevator")
    for mine in instance.mines.values():
        graph.add_node(mine.location, node_type="Mine", ore_per_hour=mine.ore_per_hour)
    other_locations = (
        set(instance.locations)
        - {instance.elevator_location}
        - {m.location for m in instance.mines.values()}
    )
    for location in other_locations:
        graph.add_node(location, node_type="Location")

    # Add edges for all tunnels (both directions)
    for tunnel in instance.tunnels:
        # Add edges in both directions with initial flow of 0
        graph.add_edge(
            tunnel.source,
            tunnel.target,
            throughput_per_hour=tunnel.throughput_per_hour,
            reinforcement_costs=tunnel.reinforcement_costs,
            flow=0,
            used=False,
        )
        graph.add_edge(
            tunnel.target,
            tunnel.source,
            throughput_per_hour=tunnel.throughput_per_hour,
            reinforcement_costs=tunnel.reinforcement_costs,
            flow=0,
            used=False,
        )

    # Update the graph with flow information from the solution
    for (u, v), utilization in solution.flow:
        if graph.has_edge(u, v):
            graph[u][v]["flow"] = utilization
            graph[u][v]["used"] = True
        else:
            print(f"Warning: Edge ({u}, {v}) not found in the instance's tunnels.")

    # Define node colors and sizes
    node_colors = []
    node_sizes = []
    for node, data in graph.nodes(data=True):
        if data["node_type"] == "Elevator":
            node_colors.append("red")
            node_sizes.append(400)
        elif data["node_type"] == "Mine":
            node_colors.append("lightblue")
            node_sizes.append(
                300 + data["ore_per_hour"] * 30
            )  # Scale size by ore_per_hour
        else:
            node_colors.append("green")
            node_sizes.append(300)

    plt.figure(figsize=(10, 10))

    # Draw nodes
    nx.draw_networkx_nodes(graph, pos, node_color=node_colors, node_size=node_sizes)

    # Edges with flow (used edges)
    flow_edges = [(u, v) for u, v, d in graph.edges(data=True) if d["used"]]
    flow_weights = [d["flow"] for u, v, d in graph.edges(data=True) if d["used"]]
    max_flow = max(flow_weights) if flow_weights else 1
    edge_widths = [2 + 3 * (w / max_flow) for w in flow_weights]

    nx.draw_networkx_edges(
        graph,
        pos,
        edgelist=flow_edges,
        width=edge_widths,
        arrowstyle="->",
        arrowsize=15,
    )

    # Edges without flow (unused edges)
    no_flow_edges = [(u, v) for u, v, d in graph.edges(data=True) if not d["used"]]
    nx.draw_networkx_edges(
        graph,
        pos,
        edgelist=no_flow_edges,
        style="dashed",
        edge_color="grey",
        arrows=False,
    )

    # Add edge labels for flow amounts
    edge_labels = {
        (u, v): f"{d['flow']}/{d['throughput_per_hour']}"
        for u, v, d in graph.edges(data=True)
        if d["used"]
    }
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=8)

    # Add node labels, including ore_per_hour for mines
    node_labels = {}
    for node, data in graph.nodes(data=True):
        if data["node_type"] == "Mine":
            node_labels[node] = f"{node}"
        else:
            node_labels[node] = str(node)
    nx.draw_networkx_labels(graph, pos, labels=node_labels, font_size=8)

    # Create a legend
    legend_elements = [
        mpatches.Patch(color="red", label="Elevator"),
        mpatches.Patch(color="lightblue", label="Mine"),
        mpatches.Patch(color="green", label="Location"),
        mpatches.Patch(color="black", label="Used Tunnel"),
        mpatches.Patch(color="grey", label="Unused Tunnel"),
    ]
    plt.legend(handles=legend_elements)

    plt.title("Solution Visualization")
    plt.axis("off")
    plt.show()

In [ ]:
# Visualize the solution using the same positions
visualize_solution(instance, solution, layout)